## Projeto - Credit Card Approval Prediction

Esse projeto será realizado através dos datasets contidos em https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction

No arquivo estão contidos dois datasets:

**credit_record**: Este data set possui 3 informações. O *ID* do indivíduo, *MONTHS_BALANCE*, que indica o último pago em relação ao mês corrente e *STATUS* que classifica o tempo de vencimento do pagamento caso tenha sido pego algum crédito, se foi pago no mês ou se não há empréstimo para o indivíduo no mês corrente.

**application_record**: Este data set possui informações pessoais referentes a cada indivíduo, que pode ser identificado através do *ID*.

Como resultado desse projeto, deve ser criado um programa no qual será possível idenitificar através do input de informações pessoais se um valor de crédito deve ou não ser aprovado para esta pessoa com base no treinamento das informações disponibilizadas. Para isso, deve ser utilizado um modelo de classificação, capaz de classificar o indivíduo entra **apto** e **não apto** a receber o empréstimo.

In [3]:
import pandas as pd

In [4]:
df_credit_record = pd.read_csv("datasets/credit_record.csv")
df_application_record = pd.read_csv("datasets/application_record.csv")

Analistando primeiro o dataset que contém as informações é possível perceber que parte das informações de cada indivíduo estão em atributos categóricos e outros em atributos numéricos. Contudo alguns valores categóricos estão em colunas classificadas como numéricas devido a sinalização da categoria ser feita de maneira binária (0-1) Felizmente, no kaggle foi disponibilizado um dicionário dos dados, descrevendo as informações contida em cada um dos atributos.

In [19]:
df_application_record.shape

(438557, 18)

In [5]:
df_application_record.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [6]:
df_application_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

É possível perceber que os atributos: *FLAG_MOBIL*, *FLAG_WORK_PHONE*, *FLAG_PHONE* e *FLAG_EMAIL* são na verdade atributos categóricos apesar de terem sido carregados automáticamente como valores inteiros.

In [15]:
# Verificando linhas duplicadas
df_application_record.duplicated().sum()

0

In [25]:
# Verificando se há duplicidade de IDs
df_application_record["ID"].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
438552    False
438553    False
438554    False
438555    False
438556    False
Name: ID, Length: 438557, dtype: bool

Foram identificados 47 IDs repetidos. Contudo, devido a grande quantidade de linhas disponíveis e diferentes informações para o mesmo ID, optarei por remover as linhas de IDs repetidas para que não tenha o input de informações incorretas sob um custo de informação baixo comparado a quantidade total de linhas na base de dados (0,0001%).

In [36]:
47/df_application_record.shape[0]

0.00010716964955524595

In [40]:
# Removendo os IDs duplicados.
df_application_record = df_application_record[df_application_record["ID"].duplicated() == False].copy()

Observando os outros atributos, *DAYS_BIRTH* e *DAYS_EMPLOYED* podem ser convertidos para valores do tipo data. Contudo, como estamos tentando realizar um input em modelo de classificação, o tempo em dias pode ser uma forma mais adequada para a utilização, não necessitando de uma transformação ao formato de data.

### Análise Exploratória dos Dados

Para iniciar a EDA, irei inicialmente separar os dados em categóricos e não categóricos, possibilitando a análise em partes. Iniciarei a EDA pelos dados não categóricos, que são a maior parte dos atributos do dataset.

In [50]:
list(df_application_record)

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS']

In [52]:
cat_attribs = [
    'ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_MOBIL',
    'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE']
num_attribs = [
 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']

In [56]:
df_ar_cat = df_application_record[cat_attribs]
df_ar_num = df_application_record[num_attribs]

#### Atributos numéricos

In [49]:
pd.options.display.float_format = "{:.2f}".format  # Alterando o formato de exibição do pandas para evitar notação científica.
df_application_record.describe()

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
count,438557.00,438557.00,438557.00,438557.00,438557.00,438557.00,438557.00,438557.00,438557.00,438557.00
mean,6022176.27,0.43,187524.29,-15997.90,60563.68,1.00,0.21,0.29,0.11,2.19
std,571637.02,0.72,110086.85,4185.03,138767.80,0.00,0.40,0.45,0.31,0.90
min,5008804.00,0.00,26100.00,-25201.00,-17531.00,1.00,0.00,0.00,0.00,1.00
25%,5609375.00,0.00,121500.00,-19483.00,-3103.00,1.00,0.00,0.00,0.00,2.00
50%,6047745.00,0.00,160780.50,-15630.00,-1467.00,1.00,0.00,0.00,0.00,2.00
75%,6456971.00,1.00,225000.00,-12514.00,-371.00,1.00,0.00,1.00,0.00,3.00
max,7999952.00,19.00,6750000.00,-7489.00,365243.00,1.00,1.00,1.00,1.00,20.00
